In [ ]:
#используемые библиотеки в решении
import pandas as pd
import lightgbm

#чтение данных
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv",index_col=0)
X_val = pd.read_csv("X_test.csv", index_col=0)

X_all = X_train_in.append(X_val, ignore_index=True)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(X_all.ДИСЦИПЛИНА)

X_train_in.loc[:,"type"] = le.transform(X_train_in.loc[:,"ДИСЦИПЛИНА"])
X_val.loc[:,"type"] = le.transform(X_val.loc[:,"ДИСЦИПЛИНА"])

X_train_in["mark"]=y_train_in["mark"]
# вычисление средней оценки
stud_mean_mark = X_train_in.groupby("STD_ID").mean()["mark"]
X_train_in.loc[:,"stud_mean"] = X_train_in.loc[:,"STD_ID"].map(stud_mean_mark)

X_val.loc[:,"stud_mean"] = X_val.loc[:,"STD_ID"].map(stud_mean_mark)

from sklearn.model_selection import train_test_split
 #разделение на обучающую и тестовую выборку для проверки решения
X_train, X_test, y_train, y_test =
train_test_split(X_train_in,X_train_in["mark"], shuffle=True, test_size=0.2)

#выбор параметров на основе которых будет обучение
train_col = ["STD_ID","КУРС","СЕМЕСТР","stud_mean","type"]


X_train["STD_ID"]= X_train["STD_ID"].astype(int)
X_test["STD_ID"]= X_test["STD_ID"].astype(int)

#создание и обучение модели на основе бустинга рещающих деревьев
lgb = lightgbm.LGBMRegressor(n_estimators=1000, verbose=1)

lgb.fit(X_train[train_col],y_train)

from sklearn.metrics import mean_absolute_error, r2_score
#вычисление метрик получившейся модели
print(mean_absolute_error(y_test, lgb.predict(X_test[train_col])))
print(r2_score(y_test, lgb.predict(X_test[train_col])))

 #сохранение решения в нужном формате для отпавки
y_pred = pd.read_csv("sample_submission.csv",index_col=0)
y_pred["mark"] = lgb.predict(X_val[train_col])

y_pred.to_csv("baseline_submission.csv")